**About :** Generates candidates.

**TODO**:

In [ ]:
cd ../src

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import gc
import sys
import cudf
import json
import glob
import pickle
import warnings
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from datetime import datetime
from collections import Counter
from pandarallel import pandarallel
from numerize.numerize import numerize

os.environ["CUDA_VISIBLE_DEVICES"] = "3"
warnings.simplefilter(action="ignore", category=FutureWarning)

pandarallel.initialize(nb_workers=32, progress_bar=False)

In [ ]:
from params import *

from data.covisitation import compute_covisitation_matrix
from data.candidates import load_parquets, create_candidates, explode, matrix_to_candids_dict

from utils.metrics import get_coverage
from utils.chris import suggest_clicks, suggest_buys, read_file_to_cache

## Covisitation matrices
- Recompute on train without using val ??

In [ ]:
MODE = "train"

In [ ]:
if MODE == "val":
    files = glob.glob("../output/full_train_parquet/*") +  glob.glob("../output/val_parquet/*")
elif MODE == "val_c":
    files = glob.glob("../output/full_train_parquet/*") +  glob.glob("../output/val_c_parquet/*")
elif MODE == "train":  # train
    files = glob.glob("../output/other_parquet/*") +  glob.glob("../output/train_parquet/*")
elif MODE == "test":
    raise NotImplementedError
else:
    raise NotImplementedError

# files = glob.glob("../input/chris/*_parquet/*")

In [ ]:
# data_cache = {}
# for f in tqdm(files):
#     data_cache[f] = read_file_to_cache(f)

In [ ]:
# mins = []
# maxs = []
# st = 0
# sv = 0
# nv = 0
# nt = 0

# for k in data_cache.keys():
#     if ("train" in k and MODE == "train") or ("val" in k) or ("test" in k):
#         mins.append(data_cache[k]['ts'].min())
#         sv += len(data_cache[k]['session'].unique())
#         nv += len(data_cache[k])
#     else:
#         maxs.append(data_cache[k]['ts'].max())
#         st += len(data_cache[k]['session'].unique())
#         nt += len(data_cache[k])

# print(f'No ts leak :', np.min(mins) > np.max(maxs))
# print(f'Train sess : {numerize(st)}')
# print(f'Train size : {numerize(nt)}')
# print(f'Val sess   : {numerize(sv)}')
# print(f'Val size   : {numerize(nv)}')

In [ ]:
# for n in [10, 20]:
#     compute_covisitation_matrix(
#         files,
#         data_cache,
#         weighting="temporal",
#         n=n,
#         save_folder="../output/matrices/",
#         suffix=MODE,
#     )

#     compute_covisitation_matrix(
#         files,
#         data_cache,
#         weighting="type",
#         type_weight={0: 1, 1: 3, 2: 6},  # 163 ??
#         n=n,
#         save_folder="../output/matrices/",
#         suffix=MODE,
#     )
    
#     compute_covisitation_matrix(
#         files,
#         data_cache,
#         considered_types=[1, 2],
#         weighting="",
#         n=n,
#         save_folder="../output/matrices/",
#         suffix=MODE,
#     )

#     compute_covisitation_matrix(
#         files,
#         data_cache,
#         weighting="type",
#         type_weight={0: 0.5, 1: 9, 2: 0.5},  # 163 ??
#         n=n,
#         save_folder="../output/matrices/",
#         suffix=MODE,
#     )

## Chris Rerank

In [ ]:
# df_val = load_parquets("../output/val_parquet/*")

# top_clicks = df_val.loc[df_val["type"] == 0, "aid"].value_counts().index.values[:20]
# top_carts = df_val.loc[df_val["type"] == 1, "aid"].value_counts().index.values[:20]
# top_orders = df_val.loc[df_val["type"] == 2, "aid"].value_counts().index.values[:20]

In [ ]:
# clicks_candids = matrix_to_candids_dict(
#     cudf.read_parquet("../output/matrices/matrix_123_temporal_20_val.pqt")
# )
# type_weighted_candids = matrix_to_candids_dict(
#     cudf.read_parquet("../output/matrices/matrix_123_type0.590.5_20_val.pqt")
# )
# cartbuy_candids = matrix_to_candids_dict(
#     cudf.read_parquet("../output/matrices/matrix_12__20_val.pqt")
# )

In [ ]:
# %%time
# pred_df_clicks = df_val.groupby(["session"]).apply(
#     lambda x: suggest_clicks(x, clicks_candids, top_clicks)
# )

In [ ]:
# %%time
# pred_df_buys = df_val.groupby(["session"]).apply(
#     lambda x: suggest_buys(x, type_weighted_candids, cartbuy_candids, top_orders)
# )

In [ ]:
# %%time

# try:
#     clicks_pred_df = pd.DataFrame(pred_df_clicks.add_suffix("_clicks"), columns=["labels"]).reset_index()
# except:
#     clicks_pred_df = pd.DataFrame(pred_df_buys.add_suffix("_clicks"), columns=["labels"]).reset_index()

# orders_pred_df = pd.DataFrame(pred_df_buys.add_suffix("_orders"), columns=["labels"]).reset_index()
# carts_pred_df = pd.DataFrame(pred_df_buys.add_suffix("_carts"), columns=["labels"]).reset_index()

# pred_df = pd.concat([clicks_pred_df, orders_pred_df, carts_pred_df])
# pred_df.columns = ["session_type", "labels_l"]
# pred_df["labels"] = pred_df["labels_l"].apply(lambda x: " ".join(map(str, x)))

In [ ]:
# gt = pd.read_parquet("../output/val_labels.parquet")

# recs = []
# df_pred = pred_df[["session_type", "labels_l"]].copy()
# df_pred.columns = ["session_type", "candidates"]
# df_pred["session"] = (
#     df_pred["session_type"].apply(lambda x: x.split("_")[0]).astype(int)
# )
# df_pred["type"] = df_pred["session_type"].apply(lambda x: x.split("_")[1])

# df_pred = df_pred.merge(gt, on=["session", "type"], how="left")

# for col in CLASSES:
#     df_pred_c = df_pred[df_pred["type"] == col]

#     n_preds, n_gts, n_found = get_coverage(
#         df_pred_c["candidates"].values, df_pred_c["ground_truth"].values
#     )
#     print(
#         f"- {col}\t- Found {numerize(n_found)} GTs with {numerize(n_preds)} candidates (pos_prop={n_found / n_preds * 100 :.2f}%)\t-  Highest reachable Recall : {n_found / n_gts :.4f}"
#     )
    
#     recs.append(n_found / n_gts)

In [ ]:
# cv = np.average(recs, weights=WEIGHTS)
# print(f"-> CV : {cv:.4f}")

- clicks	- Found 923.34K GTs with 36.03M candidates (pos_prop=2.56%)	-  Highest reachable Recall : 0.5260
- carts	- Found 235.81K GTs with 36.03M candidates (pos_prop=0.65%)	-  Highest reachable Recall : 0.4094
- orders	- Found 203.05K GTs with 36.03M candidates (pos_prop=0.56%)	-  Highest reachable Recall : 0.6482

## Candidates

### Params

In [ ]:
MODE = "train"
SUFFIX = "v2"

In [ ]:
N_MATRIX = 10
MAX_COOC = 100

In [ ]:
if MODE == "val":
    PARQUET_FILES = "../output/val_parquet/*"
elif MODE == "val_c":
    PARQUET_FILES = "../output/val_c_parquet/*"
elif MODE == "train":
    PARQUET_FILES = "../output/train_parquet/*"
elif MODE == "test":
    raise NotImplementedError
else:
    raise NotImplementedError

### Load

In [ ]:
df = load_parquets(PARQUET_FILES)

df = df.sort_values(["session", "ts"]).reset_index(drop=True)

In [ ]:
clicks_candids = matrix_to_candids_dict(
    cudf.read_parquet(f"../output/matrices/matrix_123_temporal_{N_MATRIX}_{MODE}.pqt")
)
type_weighted_candids = matrix_to_candids_dict(
    cudf.read_parquet(f"../output/matrices/matrix_123_type0.590.5_{N_MATRIX}_{MODE}.pqt")
)
# cartbuy_candidates = matrix_to_candids_dict(
#     cudf.read_parquet(f"../output/matrices/matrix_12__{n_matrix}.pqt")
# )

### Candidates

In [ ]:
%%time
df = create_candidates(df, clicks_candids, type_weighted_candids, max_cooc=MAX_COOC)

In [ ]:
del clicks_candids, type_weighted_candids
gc.collect()

### Coverage

In [ ]:
n_candid = df["candidates"].apply(len)
sns.histplot(np.clip(n_candid, 0, 150))

plt.title(f"Proportion of sessions with <20 candidates : {(n_candid < 20).mean() :.3f}")
plt.show()

In [ ]:
recalls = []
gt = pd.read_parquet(f"../output/{MODE}_labels.parquet")
# gt = pd.read_parquet("../input/chris/test_labels.parquet")

for col in CLASSES:
    if f"gt_{col}" not in df.columns:
        df = df.merge(gt[gt["type"] == col].drop("type", axis=1), how="left").rename(
            columns={"ground_truth": f"gt_{col}"}
        )

    n_preds, n_gts, n_found = get_coverage(
        df["candidates"].values, df[f"gt_{col}"].values
    )

    print(
        f"{col}\t- Found {numerize(n_found)} GTs with {numerize(n_preds)} candidates (pos_prop={n_found / n_preds * 100 :.2f}%)\t-  Highest reachable Recall : {n_found / n_gts :.3f}"
    )
    recalls.append(n_found / n_gts)

In [ ]:
cv = np.average(recalls, weights=WEIGHTS)
print(f"-> Highest reachable CV : {cv:.3f}")

### Explode & saving

In [ ]:
df = explode(df)

In [ ]:
df.to_parquet(
    f"../output/candidates/candidates_{SUFFIX}_{MODE}.parquet", index=False
)
print(f"Saved to ../output/candidates/candidates_{SUFFIX}_{MODE}.parquet")

In [ ]:
del df
gc.collect()

Done